<a href="https://colab.research.google.com/github/kabilanb7/404-Error/blob/master/Ikig_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install accelerate

In [ ]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import accelerate

model_path = "01-ai/Yi-6B-200K"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
%%time
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda", torch_dtype="auto")
model.push_to_hub("Ikig-AI")

In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.push_to_hub("Ikig-AI")

In [ ]:
%%time

user_input = "Introducing Sebastee, a young adult experiencing anxiety and stress.He connects with peers and psychiatrists using the community platform that Ikig-AI offers as soon as he opens it. Here, he can connect with people going through similar struggles and express his feelings. Sebastee finds personalised songs and music to help him relax, as well as themed courses and peaceful activities like walks through nature. By winning prizes for finishing games and workshops, he is encouraged to strive toward improving his mental health. In order to provide a flawless user experience, Ikig-AI uses cutting edge technology such as facial expression analysis and natural language processing behind the scenes for personalised recommendations and stress recognition. The program has a rewards leaderboard for friendly competition, a community page, a psychiatrist dashboard, and a personal performance dashboard."

prompt = f"""
You are Ikig-AI, an empathetic chatbot, which consoles the User, and makes his/her mood positive, joyful and enjoyable.
<Ikig-AI>: Hello! I'm here to help you with any psychological concerns or questions you may have. Feel free to share your thoughts and feelings with me, and I'll do my best to provide support and guidance.
<User>: {user_input}
<Ikig-AI>:
"""

inputs = tokenizer(prompt, return_tensors="pt")

max_length = 256

outputs = model.generate(
    inputs.input_ids.cuda(),
    max_length=max_length,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    repetition_penalty=1.3,
    no_repeat_ngram_size=5,
    temperature=0.5,
    top_k=40,
    top_p=0.8,
)

In [ ]:
%%time
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import os
os.environ['LC_ALL'] = 'C.UTF-8'

In [ ]:
os.environ['LANG'] = 'C.UTF-8'

In [ ]:
import locale

In [ ]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [ ]:
%pip install accelerate gradio

import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import accelerate

model_path = "01-ai/Yi-6B-200K"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda", torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained(model_path)

def respond_to_input(user_input):
    prompt = f"""
    You are Ikig-AI, an empathetic chatbot, which consoles the User, and makes his/her mood positive, joyful and enjoyable.
    <Ikig-AI>: Hello! I'm here to help you with any psychological concerns or questions you may have. Feel free to share your thoughts and feelings with me, and I'll do my best to provide support and guidance.
    <User>: {user_input}
    <Ikig-AI>:
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    max_length = 256
    outputs = model.generate(
        inputs.input_ids.cuda(),
        max_length=max_length,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        repetition_penalty=1.3,
        no_repeat_ngram_size=5,
        temperature=0.5,
        top_k=40,
        top_p=0.8,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

interface = gr.Interface(
    fn=respond_to_input,
    inputs=gr.Textbox(lines=5, placeholder="Type your message here..."),  # Multiline text input
    outputs="text",  # Text output
    title="Ikig-AI",  # Title of the web interface
    description="Online psychology assistant, to help your mental health."
)

interface.launch(share=True)